# Mock ETL Process for Practice


## Extract data
* Extract csv file from computer to prepare for BI Dashboard

In [1]:
import pandas as pd

def extract_data(source_path: str, source_type: str = "csv", inspect: bool = False) -> pd.DataFrame:
    """Load data from a given file path and optionally inspect it."""
    if source_type == "csv":
        df = pd.read_csv(source_path)  # Use the provided path
    else:
        raise ValueError(f"Unsupported source type: {source_type}")

    if inspect:
        print("=== Data Inspection ===")
        print(f"Shape: {df.shape}")
        print("\nFirst 5 rows:")
        print(df.head())
    
    return df

In [2]:
# call extract data function
df = extract_data("/Users/douglas/CSV/Target_Table.csv", inspect=True)  

# df1 = extract_data("/Users/douglas/CSV/Table1.csv", inspect=True) 
# df2 = extract_data("/Users/douglas/CSV/Table2.csv", inspect=True) 
# df3 = extract_data("/Users/douglas/CSV/Table3.csv", inspect=True) 


=== Data Inspection ===
Shape: (1350, 11)

First 5 rows:
  date_created  contacts_n  contacts_n_1  contacts_n_2  contacts_n_3  \
0   2022-01-15         2.0           0.0           0.0           0.0   
1   2022-01-18         3.0           1.0           1.0           0.0   
2   2022-03-06         3.0           0.0           0.0           0.0   
3   2022-01-19         3.0           1.0           0.0           1.0   
4   2022-03-18         3.0           0.0           0.0           0.0   

   contacts_n_4  contacts_n_5  contacts_n_6  contacts_n_7 new_type new_market  
0           0.0           0.0           1.0           0.0   type_1   market_3  
1           0.0           0.0           0.0           0.0   type_1   market_3  
2           1.0           0.0           0.0           0.0   type_1   market_3  
3           0.0           0.0           0.0           0.0   type_1   market_3  
4           1.0           0.0           0.0           0.0   type_1   market_3  


In [3]:
# potential to do union all in Python as part of the pipeline

# # Equivalent to UNION ALL
# union_all_df = pd.concat([df1, df2, df3], ignore_index=True)

# Transform Data:

* Rename catagorical instances in new_type and new_market
* Detect null values
* Impute 0's into null values
* Verify no nulls
* Create a total callbacks column.
* Create a callbacks to contact column

In [4]:
def missing_values_table(df):
    # calculate the sum of null values in each column    
    missing_values = df.isnull().sum()

    # calculate the percentage of nulls in each column
    percent_of_nulls = 100 * df.isnull().sum() / len(df)

    # Combine the two Series(count and percentage)
    count_perc_table = pd.concat([missing_values, percent_of_nulls], axis=1)

    # rename columns
    count_perc_col_names = count_perc_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})
    
    # Sort the data
    count_perc_col_names = count_perc_col_names[
        count_perc_col_names.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    
    # Print the shape of the dataframe
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(count_perc_col_names.shape[0]) +
          " columns that have missing values.")
    
    # Return the dataframe with missing information
    return count_perc_col_names

table = missing_values_table(df)
table

Your selected dataframe has 11 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
contacts_n_6,634,47.0
contacts_n_5,631,46.7
contacts_n_7,617,45.7
contacts_n_4,611,45.3
contacts_n_3,572,42.4
contacts_n_2,529,39.2
contacts_n_1,458,33.9
contacts_n,182,13.5


In [5]:
# impute null values as zero's

# create new dataframe
contacts_imputed = df.filter(regex='^contacts_n').fillna(0)

# Verify change in null values
table = missing_values_table(contacts_imputed)

table

Your selected dataframe has 8 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [6]:
# Calculate the total callbacks and create a new column
# the sum of contacts_imputed[n_1:n_7] minus [contacts_n]

total_callbacks = contacts_imputed.loc[:, 'contacts_n_1':'contacts_n_7'].sum(axis=1) 

# Insert total_callbacks into the contacts_imputed df
contacts_imputed.insert(8, 'total_callbacks', total_callbacks)

print(contacts_imputed.head())

   contacts_n  contacts_n_1  contacts_n_2  contacts_n_3  contacts_n_4  \
0         2.0           0.0           0.0           0.0           0.0   
1         3.0           1.0           1.0           0.0           0.0   
2         3.0           0.0           0.0           0.0           1.0   
3         3.0           1.0           0.0           1.0           0.0   
4         3.0           0.0           0.0           0.0           1.0   

   contacts_n_5  contacts_n_6  contacts_n_7  total_callbacks  
0           0.0           1.0           0.0              1.0  
1           0.0           0.0           0.0              2.0  
2           0.0           0.0           0.0              1.0  
3           0.0           0.0           0.0              2.0  
4           0.0           0.0           0.0              1.0  


In [7]:
# detect and remove illogical callbacks

false_callbacks = (contacts_imputed['total_callbacks'] > 0) & (contacts_imputed['contacts_n'] == 0)
print(false_callbacks.sum())

# correct false callbacks by setting callbacks to zero
contacts_imputed.loc[contacts_imputed['contacts_n'] == 0, 'total_callbacks'] = 0

# Verify
false_callbacks = (contacts_imputed['total_callbacks'] > 0) & (contacts_imputed['contacts_n'] == 0)
print(false_callbacks.sum())

31
0


In [8]:
import numpy as np 

# Creation of the callbacks to contact ratio

contacts_imputed['callbacks_per_contact'] = np.where(
    contacts_imputed['contacts_n'] <= 0,  # Catch negative values too
    0,
    (contacts_imputed['total_callbacks'] / contacts_imputed['contacts_n']).round()
)

print(contacts_imputed.head())

   contacts_n  contacts_n_1  contacts_n_2  contacts_n_3  contacts_n_4  \
0         2.0           0.0           0.0           0.0           0.0   
1         3.0           1.0           1.0           0.0           0.0   
2         3.0           0.0           0.0           0.0           1.0   
3         3.0           1.0           0.0           1.0           0.0   
4         3.0           0.0           0.0           0.0           1.0   

   contacts_n_5  contacts_n_6  contacts_n_7  total_callbacks  \
0           0.0           1.0           0.0              1.0   
1           0.0           0.0           0.0              2.0   
2           0.0           0.0           0.0              1.0   
3           0.0           0.0           0.0              2.0   
4           0.0           0.0           0.0              1.0   

   callbacks_per_contact  
0                    0.0  
1                    1.0  
2                    0.0  
3                    1.0  
4                    0.0  


In [10]:
print(contacts_imputed['callbacks_per_contact'].describe())

count    1350.000000
mean        0.323704
std         0.957024
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        18.000000
Name: callbacks_per_contact, dtype: float64


In [11]:
# create variables with the 3 columns needed in the new dataframe
new_data_1 = df['date_created']  # List/array for first new column
new_data_2 = df['new_type']  # List/array for second new column
new_data_3 = df['new_market'] # List/array for third new column

In [12]:
print(contacts_imputed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   contacts_n             1350 non-null   float64
 1   contacts_n_1           1350 non-null   float64
 2   contacts_n_2           1350 non-null   float64
 3   contacts_n_3           1350 non-null   float64
 4   contacts_n_4           1350 non-null   float64
 5   contacts_n_5           1350 non-null   float64
 6   contacts_n_6           1350 non-null   float64
 7   contacts_n_7           1350 non-null   float64
 8   total_callbacks        1350 non-null   float64
 9   callbacks_per_contact  1350 non-null   float64
dtypes: float64(10)
memory usage: 105.6 KB
None


In [13]:
# insert columns into the proper order in the dataframe

# Insert first column at position 0
contacts_imputed.insert(0, 'date_created', new_data_1)

# Insert second column at position 10 (or any other position)
contacts_imputed.insert(10, 'new_type', new_data_2)

# Insert third column at position 11 (or any other position)
contacts_imputed.insert(11, 'new_market', new_data_3)


# Verify
print(contacts_imputed.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date_created           1350 non-null   object 
 1   contacts_n             1350 non-null   float64
 2   contacts_n_1           1350 non-null   float64
 3   contacts_n_2           1350 non-null   float64
 4   contacts_n_3           1350 non-null   float64
 5   contacts_n_4           1350 non-null   float64
 6   contacts_n_5           1350 non-null   float64
 7   contacts_n_6           1350 non-null   float64
 8   contacts_n_7           1350 non-null   float64
 9   total_callbacks        1350 non-null   float64
 10  new_type               1350 non-null   object 
 11  new_market             1350 non-null   object 
 12  callbacks_per_contact  1350 non-null   float64
dtypes: float64(10), object(3)
memory usage: 137.2+ KB
None


In [14]:
# Rename categorical instances to make vizualizations easier to read
contacts_imputed['new_type'] = contacts_imputed['new_type'].replace({'type_1': 'account_management', 'type_2': 'technician_troubleshooting', 'type_3': 'scheduling', 'type_4': 'construction', 'type_5': 'internet_and_wifi'})

contacts_imputed['new_market'] = contacts_imputed['new_market'].replace({'market_1':'San_Francisco', 'market_2': 'Chicago', 'market_3': 'New_York'})

# Rename Columns for easier interpretation
contacts_imputed.rename(columns={'new_type': 'issue_type', 'new_market': 'market'}, inplace=True)

print(contacts_imputed.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date_created           1350 non-null   object 
 1   contacts_n             1350 non-null   float64
 2   contacts_n_1           1350 non-null   float64
 3   contacts_n_2           1350 non-null   float64
 4   contacts_n_3           1350 non-null   float64
 5   contacts_n_4           1350 non-null   float64
 6   contacts_n_5           1350 non-null   float64
 7   contacts_n_6           1350 non-null   float64
 8   contacts_n_7           1350 non-null   float64
 9   total_callbacks        1350 non-null   float64
 10  issue_type             1350 non-null   object 
 11  market                 1350 non-null   object 
 12  callbacks_per_contact  1350 non-null   float64
dtypes: float64(10), object(3)
memory usage: 137.2+ KB
None


In [15]:
print(contacts_imputed.head())

  date_created  contacts_n  contacts_n_1  contacts_n_2  contacts_n_3  \
0   2022-01-15         2.0           0.0           0.0           0.0   
1   2022-01-18         3.0           1.0           1.0           0.0   
2   2022-03-06         3.0           0.0           0.0           0.0   
3   2022-01-19         3.0           1.0           0.0           1.0   
4   2022-03-18         3.0           0.0           0.0           0.0   

   contacts_n_4  contacts_n_5  contacts_n_6  contacts_n_7  total_callbacks  \
0           0.0           0.0           1.0           0.0              1.0   
1           0.0           0.0           0.0           0.0              2.0   
2           1.0           0.0           0.0           0.0              1.0   
3           0.0           0.0           0.0           0.0              2.0   
4           1.0           0.0           0.0           0.0              1.0   

           issue_type    market  callbacks_per_contact  
0  account_management  New_York          

# Load
* Save file as a csv 
* Will be ready for upload to Tableau
* Contains additional data in table for more in depth analysis if desired

In [20]:
import os


# Save to a folder named 'output' in current working directory
output_dir = 'output'  
csv_path = os.path.join(output_dir, 'contacts_imputed2.csv') 

# Create directory 
os.makedirs(output_dir, exist_ok=True)

# Save DataFrame to CSV
contacts_imputed.to_csv(csv_path, index=False)

# Verify 
print(f"CSV saved to: {os.path.abspath(csv_path)}")

CSV saved to: /Users/douglas/output/contacts_imputed2.csv
